In [12]:
# dependencies
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import numpy as np
import getpass
import os
from datetime import datetime as dt
import glob

### Zillow Data Processing

In [13]:
listing_file = 'Resources/median_list_price.csv'
listing_df = pd.read_csv(listing_file, encoding='latin1')
listing_df.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2015-01,2015-02,2015-03,2015-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,299000.0,309000.0,325450.0,325000.0,...,399900.0,399900.0,399900.0,399900.0,399000.0,395000.0,399000.0,399950.0,399456.0,395000.0
1,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,115000.0,116900.0,118000.0,122900.0,...,186622.0,190000.0,192900.0,194500.0,193884.0,194900.0,197900.0,199000.0,199000.0,199900.0
2,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,179000.0,179999.0,185000.0,185000.0,...,272000.0,269893.5,270000.0,277700.0,275000.0,274949.5,274500.0,270000.0,270000.0,269500.0
3,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,NaN,NaN,NaN,NaN,...,1825000.0,1795000.0,1795000.0,1788000.0,1750000.0,1695000.0,1750000.0,1775000.0,1695000.0,1750000.0
4,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,315000.0,319250.0,315000.0,329000.0,...,475000.0,490000.0,499000.0,499000.0,498844.0,499000.0,512500.0,525000.0,528000.0,529000.0


In [14]:
port_listings = listing_df.loc[(listing_df['State']=='OR')]

port_listings2 = port_listings.loc[(port_listings['City']=='Portland')]

nbrhd_listings = port_listings2.drop(['State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

final_nbrhd = nbrhd_listings.rename(columns={"RegionName": "Neighborhood"})

# listings_round = nbrhd_rename['2015-01', '2015-02','2015-03'].astype('int64')

final_nbrhd.fillna(0, inplace=True)

final_nbrhd

for col in final_nbrhd.columns[2:]:
    final_nbrhd[col] = final_nbrhd[col].astype('int64')

final_nbrhd

,Neighborhood,City,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
414,Powellhurst Gilbert,Portland,195000,199900,199900,212750,218500,218000,209950,224950,...,299000,299450,300000,309947,309473,309000,304900,299998,299947,299947
444,Hazelwood,Portland,191250,189950,207000,214950,209900,215000,214500,219700,...,312500,315000,309450,316357,309900,315463,324925,324900,312900,309950
474,Centennial,Portland,207500,199974,199900,207500,215975,215000,214999,224900,...,312450,309000,310000,310000,315000,324900,319900,319900,315450,314900
528,Lents,Portland,189950,179900,189450,206950,214900,229950,230000,235000,...,344950,339900,331500,329000,339000,334950,324900,320000,320000,335000
624,Montavilla,Portland,240000,270250,239800,259900,269949,267000,269000,269900,...,371750,385000,399000,399050,389900,378950,389900,389000,380000,379250
742,Northwest,Portland,0,0,0,0,0,0,0,0,...,432000,398000,430000,425000,389900,420000,399000,369400,410950,379000
779,St.Johns,Portland,231450,239900,262500,269000,255000,247450,255000,249950,...,357000,349900,359900,375000,359500,354500,369000,350000,359900,365000
802,Cully,Portland,0,0,0,0,0,0,0,0,...,384999,379400,422700,419900,395000,391950,412450,419500,419000,419000
842,Brentwood-Darlington,Portland,199999,200000,205000,219900,233055,249000,238250,235000,...,334750,340000,349893,345000,345000,334999,339725,345000,339900,335000
907,Richmond,Portland,0,0,0,483000,499000,489000,484450,464000,...,575000,565000,582000,549999,549999,515000,524900,529900,534900,525000


In [15]:
house_t = final_nbrhd.T

In [16]:
house_t.columns = house_t.iloc[0]
house_t

Neighborhood,Powellhurst Gilbert,Hazelwood,Centennial,Lents,Montavilla,Northwest,St.Johns,Cully,Brentwood-Darlington,Richmond,...,Arbor Lodge,Overlook,Goose Hollow,Roseway,Southwest Hills,Pearl District,Woodlawn,Northwest Heights,Hillside,Hayden Island
Neighborhood,Powellhurst Gilbert,Hazelwood,Centennial,Lents,Montavilla,Northwest,St.Johns,Cully,Brentwood-Darlington,Richmond,...,Arbor Lodge,Overlook,Goose Hollow,Roseway,Southwest Hills,Pearl District,Woodlawn,Northwest Heights,Hillside,Hayden Island
City,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland,...,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland,Portland
2015-01,195000,191250,207500,189950,240000,0,231450,0,199999,0,...,0,0,0,0,0,539000,0,695000,0,0
2015-02,199900,189950,199974,179900,270250,0,239900,0,200000,0,...,0,0,0,0,0,495000,0,725000,0,0
2015-03,199900,207000,199900,189450,239800,0,262500,0,205000,0,...,0,0,0,0,0,497000,0,685000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07,309000,315463,324900,334950,378950,420000,354500,391950,334999,515000,...,495000,472000,372000,424450,875000,555000,511950,762950,900000,374000
2019-08,304900,324925,319900,324900,389900,399000,369000,412450,339725,524900,...,509450,502450,379900,445000,898500,575000,500000,759000,880000,369000
2019-09,299998,324900,319900,320000,389000,369400,350000,419500,345000,529900,...,499450,499850,378700,435000,929000,537550,499000,770000,875000,369000
2019-10,299947,312900,315450,320000,380000,410950,359900,419000,339900,534900,...,489900,429450,377900,419900,927500,525000,499900,735000,950000,385000


In [17]:
t_house = house_t.iloc[2:].reset_index().rename(columns={"index":"Date"})
t_house

Neighborhood,Date,Powellhurst Gilbert,Hazelwood,Centennial,Lents,Montavilla,Northwest,St.Johns,Cully,Brentwood-Darlington,...,Arbor Lodge,Overlook,Goose Hollow,Roseway,Southwest Hills,Pearl District,Woodlawn,Northwest Heights,Hillside,Hayden Island
0,2015-01,195000,191250,207500,189950,240000,0,231450,0,199999,...,0,0,0,0,0,539000,0,695000,0,0
1,2015-02,199900,189950,199974,179900,270250,0,239900,0,200000,...,0,0,0,0,0,495000,0,725000,0,0
2,2015-03,199900,207000,199900,189450,239800,0,262500,0,205000,...,0,0,0,0,0,497000,0,685000,0,0
3,2015-04,212750,214950,207500,206950,259900,0,269000,0,219900,...,0,0,0,0,0,465000,0,720000,0,0
4,2015-05,218500,209900,215975,214900,269949,0,255000,0,233055,...,0,0,0,0,0,465000,0,660000,0,0
5,2015-06,218000,215000,215000,229950,267000,0,247450,0,249000,...,0,0,0,0,0,447000,0,654950,0,0
6,2015-07,209950,214500,214999,230000,269000,0,255000,0,238250,...,0,0,0,0,0,459900,0,699450,0,0
7,2015-08,224950,219700,224900,235000,269900,0,249950,0,235000,...,0,0,0,0,0,459950,0,749000,0,0
8,2015-09,219000,225000,225000,225500,275000,0,255000,0,239900,...,0,0,0,0,0,469950,0,789000,0,0
9,2015-10,219900,229900,225000,229950,297450,0,252500,0,249950,...,0,0,0,0,0,500000,0,774500,0,0


In [18]:
for col in t_house.columns[1:]:
    t_house[col] = t_house[col].astype('int64')

t_house['Date'] = pd.to_datetime(t_house['Date'])

t_house.dtypes

Neighborhood
Date                             datetime64[ns]
Powellhurst Gilbert                       int64
Hazelwood                                 int64
Centennial                                int64
Lents                                     int64
Montavilla                                int64
Northwest                                 int64
St.Johns                                  int64
Cully                                     int64
Brentwood-Darlington                      int64
Richmond                                  int64
Downtown                                  int64
Sellwood-Moreland                         int64
Portsmouth                                int64
Pleasant Valley                           int64
Mount Tabor                               int64
Mill Park                                 int64
Concordia                                 int64
Rose City Park                            int64
Wilkes                                    int64
Woodstock                  

In [19]:
new_final_house = pd.DataFrame(t_house.set_index("Date").T.mean()).reset_index().rename(columns={0:"mean_list_price"})

In [20]:
new_final_house

,Date,mean_list_price
0,2015-01-01,109031.916667
1,2015-02-01,110413.138889
2,2015-03-01,128470.805556
3,2015-04-01,165694.444444
4,2015-05-01,163570.250000
5,2015-06-01,164999.305556
6,2015-07-01,169411.083333
7,2015-08-01,169616.666667
8,2015-09-01,175218.055556
9,2015-10-01,178079.861111


In [21]:
renting_file = 'Resources/zillow_neighborhood_rent.csv'
renting_df = pd.read_csv(renting_file, encoding='latin1')
renting_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2010-09,2010-10,2010-11,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,1070.05,1076.02,1088.45,...,1364.67,1361.97,1363.92,1368.71,1372.82,1375.39,1376.78,1379.89,1396.27,1403.48
1,112345,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,878.54,897.82,895.75,...,1163.25,1163.28,1166.00,1170.93,1179.64,1190.45,1201.33,1208.60,1224.04,1234.80
2,192689,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,1093.87,1095.46,1100.74,...,1303.40,1303.82,1307.68,1315.92,1324.92,1334.68,1344.17,1350.86,1370.02,1374.16
3,270958,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,3301.80,3340.97,3347.80,...,NaN,4107.81,4095.74,4086.67,4078.03,4070.69,4042.08,4066.10,4156.94,4205.98
4,118208,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,1781.15,1807.14,1804.85,...,2395.96,2391.67,2386.86,2387.62,2393.16,2400.53,2403.38,2399.27,2395.09,2405.24


In [22]:
renting_df = renting_df.loc[(renting_df['State']=='OR')]

renting_df = renting_df.loc[(renting_df['City']=='Portland')]

rental_listings = renting_df.drop(['State', 'Metro', 'CountyName', 'SizeRank', 'RegionID'], axis=1)

rental_rename = rental_listings.rename(columns={"RegionName": "Neighborhood"})

rental_rename

,Neighborhood,City,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
478,Powellhurst Gilbert,Portland,1225.67,NaN,1150.29,1124.15,NaN,1059.43,1032.82,1009.01,...,1538.71,1522.98,1506.70,1493.02,1485.12,1485.88,1495.84,1509.54,NaN,1590.57
512,Hazelwood,Portland,NaN,1199.13,NaN,1133.08,1108.12,1084.97,1069.87,1056.22,...,1481.01,1474.52,1463.86,1453.34,1445.49,1442.18,1445.47,1454.76,NaN,1556.95
545,Centennial,Portland,NaN,1173.41,1145.02,1125.68,1094.13,1072.88,1053.71,1028.25,...,1579.26,1568.53,1558.58,1548.53,1538.92,1524.36,1525.49,1538.96,NaN,1647.89
612,Lents,Portland,1167.86,1154.11,1137.36,1117.83,NaN,1052.04,NaN,NaN,...,1559.12,1549.46,1539.62,1533.38,1529.44,1529.78,1538.55,1546.39,NaN,NaN
739,Montavilla,Portland,1159.17,1188.83,1190.45,1179.39,NaN,NaN,NaN,1031.16,...,1615.09,1599.09,1586.26,1575.19,1569.56,1569.82,1573.68,1577.01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,Maplewood,Portland,1384.84,1412.88,1425.33,1432.01,1403.05,NaN,NaN,NaN,...,1727.50,1705.80,1685.18,1677.90,1673.88,1665.53,1662.99,1678.30,NaN,1782.06
5261,Hillside,Portland,1523.28,1554.27,1566.90,1542.74,1572.19,1572.14,1592.95,1608.43,...,2324.79,2314.29,2277.20,2253.70,2262.80,2258.71,2256.26,2276.50,2331.88,2275.46
5279,Arnold Creek,Portland,NaN,1755.64,1752.31,1720.58,1679.68,1631.70,1618.05,NaN,...,1916.52,1874.22,1854.92,1891.26,1931.35,NaN,NaN,1928.16,1961.53,1911.65
5408,Markham,Portland,1521.60,1548.03,1531.81,1525.59,1488.98,NaN,NaN,NaN,...,1686.31,1678.04,1667.77,1670.05,1668.02,1709.74,1731.52,1747.72,1758.79,1780.26


In [23]:
cols = rental_rename.columns[:2].tolist()
cols_df = cols.append(rental_rename.columns[54:].tolist())

In [24]:
cols = rental_rename.columns.tolist()

In [25]:
new_cols = cols[:2] + cols[54:]

In [26]:
new_rental = rental_rename[new_cols]

In [27]:
final_rental = new_rental.fillna(0)

for col in new_rental.columns[2:]:
    final_rental[col] = final_rental[col].astype('int64')

final_rental.head

<bound method NDFrame.head of              Neighborhood      City  2015-01  2015-02  2015-03  2015-04  \
478   Powellhurst Gilbert  Portland     1247     1250     1262     1276   
512             Hazelwood  Portland     1223     1220     1228     1240   
545            Centennial  Portland     1268     1271     1282     1291   
612                 Lents  Portland     1256     1256     1266     1271   
739            Montavilla  Portland     1327     1336     1352     1363   
...                   ...       ...      ...      ...      ...      ...   
5242            Maplewood  Portland     1408     1398     1426     1456   
5261             Hillside  Portland     2033     2037     2042     2079   
5279         Arnold Creek  Portland     1662     1674     1713        0   
5408              Markham  Portland     1396     1425     1466        0   
5798        Hayden Island  Portland     1595     1607     1609     1605   

      2015-05  2015-06  2015-07  2015-08  ...  2019-02  2019-03  2019

In [28]:
df_t = final_rental.T

In [29]:
df_t.columns = df_t.iloc[0]

In [30]:
t_rental = df_t.iloc[2:].reset_index().rename(columns={"index":"Date"})

In [31]:
for col in t_rental.columns[1:]:
    t_rental[col] = t_rental[col].astype('int64')

t_rental.dtypes

Neighborhood
Date                   object
Powellhurst Gilbert     int64
Hazelwood               int64
Centennial              int64
Lents                   int64
                        ...  
Maplewood               int64
Hillside                int64
Arnold Creek            int64
Markham                 int64
Hayden Island           int64
Length: 74, dtype: object

In [32]:
t_rental['Date'] = pd.to_datetime(t_rental['Date'])
t_rental.dtypes

Neighborhood
Date                   datetime64[ns]
Powellhurst Gilbert             int64
Hazelwood                       int64
Centennial                      int64
Lents                           int64
                            ...      
Maplewood                       int64
Hillside                        int64
Arnold Creek                    int64
Markham                         int64
Hayden Island                   int64
Length: 74, dtype: object

In [33]:
new_final_rental = pd.DataFrame(t_rental.set_index("Date").T.mean()).reset_index().rename(columns={0:"mean_rental_price"})

In [34]:
p = getpass.getpass(prompt="Password: ")
rds_connection_string = f"postgres:{p}@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Password: ········


In [35]:
engine.table_names()

['airbnb_reviews',
 'airbnb_portland_mean_price',
 'airbnb_price_by_day_of_week',
 'mean_airbnb_prices',
 'airbnb_reviews_by_day',
 'airbnb_reviews_by_day_of_week',
 'airbnb_portland',
 'airbnb_room_tpyes',
 'rentals',
 'listings',
 'airbnb_mean_prices']

In [36]:
new_final_house.to_sql(name='listings', con=engine, if_exists='replace', index=False)

In [37]:
new_final_rental.to_sql(name='rentals', con=engine, if_exists='replace', index=False)

In [38]:
pd.read_sql_query('select * from listings', con=engine).head()

,Date,mean_list_price
0,2015-01-01,109031.916667
1,2015-02-01,110413.138889
2,2015-03-01,128470.805556
3,2015-04-01,165694.444444
4,2015-05-01,163570.250000


In [39]:
pd.read_sql_query('select * from rentals', con=engine).head()

,Date,mean_rental_price
0,2015-01-01,1508.301370
1,2015-02-01,1514.931507
2,2015-03-01,1526.547945
3,2015-04-01,1444.863014
4,2015-05-01,1442.849315


### Airbnb Data Proessing

In [40]:
airbnb_files = []
for file in os.listdir("Resources"):
    if file.startswith("listing"):
        airbnb_files.append(file)
cols = ["date","id","name","neighbourhood","latitude","longitude","room_type","price","number_of_reviews","reviews_per_month"]

In [41]:
df = pd.DataFrame()
for file in airbnb_files:
    date = (file.split("(")[1].split(")")[0])
    temp = pd.read_csv("Resources/"+file)[cols[1:]]
    temp["date"] = dt.strptime(date, "%b %d %Y").strftime("%Y-%m-%d")
    df = pd.concat([df,temp[cols]])
    

In [42]:
df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2016-04-05,689130,Exquisite private room w/sauna,Sullivan's Gulch,45.534838,-122.649000,Private room,100,54,1.25
1,2016-04-05,5794291,Private suites near it all- Room #6,Sullivan's Gulch,45.533373,-122.648236,Private room,100,34,3.71
2,2016-04-05,10885290,LOCATION NE : Queen Bunk Bed-TOP,Sullivan's Gulch,45.534529,-122.647962,Shared room,40,9,4.09
3,2016-04-05,4918034,Private suites near it all- Room #2,Sullivan's Gulch,45.533280,-122.647862,Private room,100,55,3.85
4,2016-04-05,5134256,Private suites near it all- Room #4,Sullivan's Gulch,45.533274,-122.646870,Private room,70,68,4.93


In [43]:
df.to_csv("data/All_airbnb_listings.csv",sep=",",index=False)

In [44]:
df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2016-04-05,689130,Exquisite private room w/sauna,Sullivan's Gulch,45.534838,-122.649000,Private room,100,54,1.25
1,2016-04-05,5794291,Private suites near it all- Room #6,Sullivan's Gulch,45.533373,-122.648236,Private room,100,34,3.71
2,2016-04-05,10885290,LOCATION NE : Queen Bunk Bed-TOP,Sullivan's Gulch,45.534529,-122.647962,Shared room,40,9,4.09
3,2016-04-05,4918034,Private suites near it all- Room #2,Sullivan's Gulch,45.533280,-122.647862,Private room,100,55,3.85
4,2016-04-05,5134256,Private suites near it all- Room #4,Sullivan's Gulch,45.533274,-122.646870,Private room,70,68,4.93


In [45]:
sorted_airbnb = df.sort_values(
    ["id", "date"], ascending=True).reset_index(drop=True)
sorted_airbnb.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2015-03-01,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
1,2015-05-12,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
2,2016-01-01,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.03
3,2017-10-04,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
4,2017-11-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02


In [46]:
reviews = 0
current_id = 0
review_col = []
for i,row in sorted_airbnb.iterrows():
    if sorted_airbnb["id"].iloc[i] != current_id:
        current_id = sorted_airbnb["id"].iloc[i]
        reviews = sorted_airbnb["number_of_reviews"].iloc[i]
    else:
        reviews = sorted_airbnb["number_of_reviews"].iloc[i] - sorted_airbnb["number_of_reviews"].iloc[i-1]       
    
    review_col.append(reviews)

In [47]:
sorted_airbnb['reviews'] = review_col

In [48]:
sorted_airbnb.loc[sorted_airbnb['reviews'] < 0] =0
sorted_airbnb['date'] = pd.to_datetime(sorted_airbnb['date'])
clean_data = sorted_airbnb.loc[sorted_airbnb['date'] > '2015-03-01']
clean_data.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month,reviews
1,2015-05-12,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00,0
2,2016-01-01,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.03,0
3,2017-10-04,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02,0
4,2017-11-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02,0
5,2017-12-09,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02,0


In [49]:
review_df= pd.DataFrame(clean_data.groupby(['date'])['reviews'].mean()).reset_index()
review_df.head()


,date,reviews
0,2015-05-12,5.311962
1,2015-09-02,10.238521
2,2015-11-02,5.264074
3,2015-12-02,1.803590
4,2016-01-01,1.341874


In [50]:
review_df.to_sql("airbnb_reviews",con=engine,index=False, if_exists="replace")

In [51]:
price_data = sorted_airbnb.loc[(sorted_airbnb['price']<1000) &(sorted_airbnb['price']>1)]

In [52]:
price= pd.DataFrame(price_data.groupby(['date'])['price'].mean()).reset_index()
price.head()


,date,price
0,2015-03-01,100.589888
1,2015-05-12,107.492823
2,2015-09-02,116.866558
3,2015-11-02,111.262417
4,2015-12-02,108.610211


In [53]:
price.to_sql("airbnb_portland_mean_price",con=engine,index=False, if_exists="replace")

In [54]:
DoW_df = sorted_airbnb

In [55]:
DoW_df['weekday'] = DoW_df['date'].dt.day_name()
DoW_df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month,reviews,weekday
0,2015-03-01,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00,2,Sunday
1,2015-05-12,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00,0,Tuesday
2,2016-01-01,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.03,0,Friday
3,2017-10-04,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02,0,Wednesday
4,2017-11-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02,0,Monday


In [56]:
review_day= pd.DataFrame(DoW_df.groupby(['weekday'])['price'].mean()).reset_index()
review_day.head()


,weekday,price
0,Friday,118.157547
1,Monday,117.443329
2,Saturday,128.580737
3,Sunday,114.909278
4,Thursday,112.909758


In [57]:
review_day.to_sql("airbnb_price_by_day_of_week",con=engine,index=False, if_exists="replace")

In [58]:
room_type= pd.DataFrame(sorted_airbnb.groupby(['room_type'])['price'].mean()).reset_index()
room_type

,room_type,price
0,0,0.000000
1,Entire home/apt,141.751356
2,Hotel room,106.758621
3,Private room,72.066119
4,Shared room,63.107887


In [59]:
last_month = sorted_airbnb.loc[sorted_airbnb['date'] == '2019-11-09'].reset_index(drop=True)
last_month.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month,reviews,weekday
0,2019-11-09,12899,"Alberta Arts 2 bedroom suite, charming 1906 house",Concordia,45.56401,-122.63472,Entire home/apt,65,535,4.49,2,Saturday
1,2019-11-09,16688,"Beautiful condo in downtown Pearl, NW Portland",Pearl,45.52542,-122.68557,Entire home/apt,275,75,2.44,3,Saturday
2,2019-11-09,29931,Lovely SW Victorian w/Bonus Room and Hot Tub,Hayhurst,45.48278,-122.72089,Entire home/apt,200,53,0.46,0,Saturday
3,2019-11-09,37676,Mt. Hood View in the Pearl District,Pearl,45.52555,-122.68193,Entire home/apt,125,123,1.09,1,Saturday
4,2019-11-09,39938,Friendly Guesthouse / 10 Min to Airport & Down...,Woodlawn,45.57428,-122.64594,Private room,29,273,4.81,0,Saturday


In [60]:
last_month.to_sql("airbnb_portland",con=engine,index=False, if_exists="replace")

In [61]:
# engine.execute("SELECT * FROM airbnb_portland").fetchall()

In [62]:
mean_airbnb = sorted_airbnb[['date','neighbourhood','price']].groupby(["neighbourhood","date"]).mean().reset_index()

In [63]:
# total_mean = mean_airbnb.iloc[1:].groupby("date").mean().reset_index()

In [64]:
# pivot_airbnb = mean_airbnb.pivot(index="neighbourhood", columns="date", values="price").fillna(0)

In [65]:
# total_mean.to_sql("airbnb_mean_prices",con=engine,index=False, if_exists="replace")

In [66]:
count_df = last_month[['neighbourhood','room_type','price']].groupby(['neighbourhood','room_type']).count()\
.reset_index().rename(columns={"price":"count"})

In [67]:
total_df = last_month[['neighbourhood','price']].groupby(['neighbourhood']).count()\
.reset_index().rename(columns={"price":"total"})

In [68]:
new_df = pd.merge(count_df,total_df,how='left',on='neighbourhood')
new_df['percent'] = new_df['count'] / new_df['total']

In [69]:
hoods = last_month[['neighbourhood','latitude','longitude']].groupby(['neighbourhood']).mean()

In [70]:
new_df = pd.merge(new_df,hoods,how='left',on='neighbourhood')

In [71]:
new_df.to_sql("airbnb_room_tpyes",con=engine,index=False, if_exists="replace")